# 작업 1유형

## 각 구급 보고서 별 출동시각과 신고시각의 차이를 ‘소요시간’ 컬럼을 만들고 초(sec)단위로 구하고 소방서명 별 소요시간의 평균을 오름차순으로 정렬 했을때 3번째로 작은 소요시간의 값과 소방서명을 출력하라


데이터 설명 : 소방신고별 신고시각과 출동시각 데이터 컬럼 설명 : 신고일자 (yyyymmdd) / 신고시각 (HHMMSS , 00시 , 00분의 경우 등 생략)

In [60]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e6_p1_1.csv')
df.head(5)

,구급보고서번호,소방서명,환자증상구분명1,신고일자,신고시각,출동일자,출동시각,환자직업명,사고발생장소
0,20211122103M01526,중랑소방서,심계항진,20210330,104800,20210330,105042,무직,집
1,20211111105M00771,노원소방서,복통,20210412,225600,20210412,225827,기타,집
2,20211115104M00864,서초소방서,심계항진,20210303,194100,20210303,194500,직장인,상업시설
3,20211113107M01869,마포소방서,고열,20211225,235400,20211226,331,유아,집
4,20211118102M05820,강동소방서,고열,20211019,12100,20211019,12322,무직,집


In [65]:
df['소요시간'] = (
                pd.to_datetime(
                    df['출동일자'].astype('str') + df['출동시각'].astype('str').str.zfill(6)
                )

                -

                pd.to_datetime(
                    df['신고일자'].astype('str') + df['신고시각'].astype('str').str.zfill(6)
                )
                ).dt.total_seconds()

res_gb = df.groupby(['소방서명'])['소요시간'].mean().sort_values().reset_index()
res = res_gb.iloc[2]
print(res)

소방서명    종로소방서
소요시간    175.5
Name: 2, dtype: object


## 학교 세부유형이 일반중학교인 학교들 중 일반중학교 숫자가 2번째로 많은 시도의 일반중학교 데이터만 필터하여 해당 시도의 교원 한명 당 맡은 학생수가 가장 많은 학교를 찾아서 해당 학교의 교원수를 출력하라

In [66]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e6_p1_2.csv')
df.head(5)

,학교명,시도,학교세부유형,일반학급_학생수_계,교원수_총계_계
0,서울대학교사범대학부설고등학교,서울,일반고등학교,689,73
1,가락고등학교,서울,일반고등학교,765,70
2,개포고등학교,서울,일반고등학교,684,69
3,경기고등학교,서울,일반고등학교,1047,96
4,경동고등학교,서울,일반고등학교,529,58


In [75]:
df = df[df['학교세부유형'] == '일반중학교']
area = df['시도'].value_counts().sort_values(ascending = False).index[1]

res_df = df[df['시도'] == area]
res_df['val'] = res_df['일반학급_학생수_계'] / res_df['교원수_총계_계']
res_df = res_df.sort_values('val', ascending = False)
res = res_df.reset_index(drop = True).loc[0, '교원수_총계_계']
print(res)

33


<ipython-input-75-2d3f104d3302>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res_df['val'] = res_df['일반학급_학생수_계'] / res_df['교원수_총계_계']


## **5대 범죄(절도, 사기, 배임, 방화, 폭행)의 월별 총 발생건수를 총범죄수라고 표현하자. 18,19년의 각각 분기별 총범죄수의 월평균 값을 구했을때 최대값을 가지는 년도와 분기를 구하고 해당 분기의 최댓값의 사기가 발생한 월의 사기 발생건수를 출력하라(1분기:1,2,3월 / 2분기 : 4,5,6월 / 3분기 7,8,9월 / 4분기 10,11,12월 , 1분기 월평균 : 1,2,3월의 총범죄수 평균) **

In [105]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e6_p1_3.csv')
df.head(5)

,범죄분류,절도,사기,배임,방화,폭행
0,2018년_1월,13280,25318,656,143,12186
1,2018년_2월,12504,19159,403,135,12087
2,2018년_3월,14733,24509,471,148,13841
3,2018년_4월,14781,23147,414,135,13973
4,2018년_5월,15988,23584,451,127,14117


In [125]:
def label(x):
    year = x.split('_')[0]
    month = x.split('_')[1]
    if month in ['1월', '2월', '3월']:
        return '1분기'
    elif month in ['4월', '5월', '6월']:
        return '2분기'
    elif month in ['7월', '8월', '9월']:
        return '3분기'
    else:
        return '4분기'
res_df = df.copy()
res_df['총범죄수'] = res_df.loc[:,'절도' : '폭행'].sum(axis = 1)
res_df['분기'] = res_df['범죄분류'].apply(label)
res_df['year'] = res_df['범죄분류'].str.split('_').str[0]
res_df['month'] = res_df['범죄분류'].str.split('_').str[1]

res_gb = res_df.groupby(['year', '분기'])['총범죄수'].mean().sort_values(ascending = False)
m_year = res_gb.index[0][0]
m_p = res_gb.index[0][1]
res = res_df[(res_df['year'] == m_year) & (res_df['분기'] == m_p)].sort_values('총범죄수', ascending = False).reset_index(drop = 0)
res_lst = res.loc[:,'절도' : '폭행'].values
print(res_lst.max())

27766


# 작업 2유형

예측 변수 General_Health, test.csv에 대해 ID별로 General_Health 값을 예측하여 제출, 제출 데이터 컬럼은 ID와 General_Health 두개만 존재해야함. 평가지표는 f1score

In [131]:
import pandas as pd
train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/ep6_p2_train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/ep6_p2_test.csv')

display(train.head(2))
test.head(2)

,ID,General_Health,Checkup,Exercise,Heart_Disease,Skin_Cancer,Other_Cancer,Depression,Diabetes,Arthritis,Sex,Age_Category,Height_(cm),Weight_(kg),BMI,Smoking_History,Alcohol_Consumption,Fruit_Consumption,Green_Vegetables_Consumption,FriedPotato_Consumption
0,ID_1473,Good,Within the past 2 years,Yes,No,No,No,No,No,Yes,Female,55-59,168.0,77.11,27.44,No,1.0,12.0,4.0,4.0
1,ID_2045,Good,Within the past 2 years,Yes,No,No,No,No,No,No,Female,40-44,152.0,58.97,25.39,No,0.0,8.0,16.0,8.0


,ID,Checkup,Exercise,Heart_Disease,Skin_Cancer,Other_Cancer,Depression,Diabetes,Arthritis,Sex,Age_Category,Height_(cm),Weight_(kg),BMI,Smoking_History,Alcohol_Consumption,Fruit_Consumption,Green_Vegetables_Consumption,FriedPotato_Consumption
0,ID_14547,Within the past year,Yes,No,No,No,No,No,No,Male,35-39,180.0,81.65,25.10,No,15.0,8.0,25.0,12.0
1,ID_26,5 or more years ago,No,No,Yes,No,No,No,No,Male,60-64,183.0,79.38,23.73,No,1.0,4.0,4.0,4.0


## 방법 1. onehotencoder

In [153]:
train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/ep6_p2_train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/ep6_p2_test.csv')


col_drop = ['ID']
col_y = ['General_Health']
test_id = test['ID']
col_num = ['Height_(cm)', 'Weight_(kg)', 'BMI', 'Alcohol_Consumption', 'Fruit_Consumption', 'Green_Vegetables_Consumption', 'FriedPotato_Consumption']
col_cat = list(set(train.columns) - set(col_y) - set(col_drop) - set(col_num))

train = train.drop(col_drop, axis = 1)
x_train = train.loc[:, 'Checkup':'FriedPotato_Consumption']
y_train = train[col_y]
x_test = test.drop(col_drop, axis = 1)

x = pd.concat([x_train, x_test])
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(handle_unknown = 'ignore')
ohe.fit(x[col_cat])

x_train_res = ohe.transform(x_train[col_cat])
x_test_res = ohe.transform(x_test[col_cat])

x_train_ohe = pd.DataFrame(x_train_res.todense(), columns = ohe.get_feature_names_out())
x_test_ohe = pd.DataFrame(x_test_res.todense(), columns = ohe.get_feature_names_out())

x_train_fin = pd.concat([x_train_ohe, x_train[col_num]], axis = 1)
x_test_fin = pd.concat([x_test_ohe, x_test[col_num]], axis = 1)

from sklearn.model_selection import train_test_split
x_tr, x_val, y_tr, y_val = train_test_split(x_train_fin, y_train.values.ravel(), test_size = 0.3, stratify = y_train.values.ravel())

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(x_tr[col_num])

x_tr[col_num] = scaler.transform(x_tr[col_num])
x_val[col_num] = scaler.transform(x_val[col_num])
x_test_fin[col_num] = scaler.transform(x_test_fin[col_num])

from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(x_tr, y_tr)

from sklearn.metrics import f1_score
y_tr_pred = model.predict(x_tr)
y_val_pred = model.predict(x_val)

print('f1 tr : ', f1_score(y_tr, y_tr_pred, average = 'macro'))
print('f1 val : ', f1_score(y_val, y_val_pred, average = 'macro'))

y_pred = model.predict(x_test_fin)
result = pd.DataFrame({'ID' : test_id, 'pred' : y_pred})
result.to_csv('00300.csv', index = False)

f1 tr :  1.0
f1 val :  0.5526116515108455


## 방법 2. dummies

In [3]:
import pandas as pd
train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/ep6_p2_train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/ep6_p2_test.csv')


col_drop = ['ID']
col_y = ['General_Health']
test_id = test['ID']
col_num = ['Height_(cm)', 'Weight_(kg)', 'BMI', 'Alcohol_Consumption', 'Fruit_Consumption', 'Green_Vegetables_Consumption', 'FriedPotato_Consumption']
col_cat = list(set(train.columns) - set(col_y) - set(col_drop) - set(col_num))

train = train.drop(col_drop, axis = 1)
x_train = train.loc[:, 'Checkup':'FriedPotato_Consumption']
y_train = train[col_y]
x_test = test.drop(col_drop, axis = 1)

x_train_dumm = pd.get_dummies(x_train)
x_test_dumm = pd.get_dummies(x_test)
x_test_dumm = x_test_dumm.reindex(columns = x_train_dumm.columns, fill_value = 0)

from sklearn.model_selection import train_test_split
x_tr, x_val, y_tr, y_val = train_test_split(x_train_dumm, y_train.values.ravel(), test_size = 0.3, stratify = y_train.values.ravel())

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(x_tr[col_num])

x_tr[col_num] = scaler.transform(x_tr[col_num])
x_val[col_num] = scaler.transform(x_val[col_num])
x_test_dumm[col_num] = scaler.transform(x_test_dumm[col_num])

from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(x_tr, y_tr)

from sklearn.metrics import f1_score
y_tr_pred = model.predict(x_tr)
y_val_pred = model.predict(x_val)

print('f1 tr : ', f1_score(y_tr, y_tr_pred, average = 'macro'))
print('f1 val : ', f1_score(y_val, y_val_pred, average = 'macro'))

y_pred = model.predict(x_test_dumm)
result = pd.DataFrame({'ID' : test_id, 'pred' : y_pred})
result.to_csv('00300.csv', index = False)

f1 tr :  1.0
f1 val :  0.534398872194867


# 작업 3유형

## A 도시의 남성 600명과 여성 550명이 있다. 남성들 중 흡연자 비율은 0.2이며 여성들 중 흡연자 비율은 0.26이다.
남성과 여성 간에 흡연 여부에 따른 인구 비율이 다른지 확인하고 싶다. 유의 수준 0.05하 귀무가설에 대해 기각 / 채택 여부와 p-value값을 각각 출력하라

In [17]:
from scipy.stats import chi2_contingency
import numpy as np

men = 600
women = 550

men_ok = int(600 * 0.2)
women_ok = int(550 * 0.26)

men_no = int(men - men_ok)
women_no = int(women - women_ok)

data = np.array([[men_ok, men_no], [women_ok, women_no]])
t, p, _, _ = chi2_contingency(data)
print(p)
if p < 0.05:
    print('기각')
else:
    print('채택')

0.018786854975740768
기각


## 연령, 몸무게,콜레스테롤 수치 데이터

In [18]:
import pandas as pd
df= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/adp/28/p7.csv')
df.head()


,age,Cholesterol,weight
0,65,69.0,111.0
1,54,117.0,81.0
2,61,86.2,72.0
3,57,76.0,78.0
4,62,160.0,61.0


## age와 Cholesterol을 가지고 weight를 예측하는 선형 회귀 모델을 만들려고한다. age의 회귀 계수를 구하여라

## age가 고정일 때 Cholesterol와 weight가 선형관계에 있다는 가설을 유의수준 0.05하에 검정하라

## age가 55, Cholesterol가 72.6일때 위 모델을 기반으로 weight값을 예측하라.